# Incorrect Column Type Prediction

In [ ]:
# Preamble to run notebook in context of source package.
# NBVAL_IGNORE_OUTPUT
import sys
sys.path.insert(0, '../')

In [ ]:
from IPython.core.display import display
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcdefaults()
import numpy as np
import pandas as pd

from ptype.Ptype import Ptype
from utils import plot_column_type_posterior, plot_arff_type_posterior, subsample_df

In [ ]:
# df = pd.read_csv('../data/grub-damage.csv', encoding="ISO-8859-1",dtype='str')
# df.head()

In [ ]:
# df['year'] = df['year'].apply(lambda x: "".join(['1', '9'] + list(x)))
# df.head()

In [ ]:
x = ['1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '90']
column = 'year'

In [ ]:
df = pd.DataFrame(x, dtype='str', columns=[column])
df

In [ ]:
ptype = Ptype()

ptype.fit_schema(df)
ptype.show_schema()

In [ ]:
ptype.cols[column]

In [ ]:
ptype.reclassify_column(column, 'date-iso-8601')

In [ ]:
ptype.cols[column]

In [ ]:
plot_column_type_posterior(ptype.cols[column].p_t)

In [ ]:
arff_type = ptype.cols[column].arff_type

plot_arff_type_posterior(ptype.cols[column].arff_posterior)

In [ ]:
# ptype.reclassify_column(column, 'string')

# plot_column_type_posterior(ptype.cols[column].p_t)

# # do the same thing for arff type